# Chatbot + Recomanació

L'objectiu d'aquesta pràctica és força directe, volem recomanar a l'usuari noves `aisle_id` on anar a comprar quan ens ho demani.

Es dóna tant un chatbot funcional com un recomanador basat en factorització de matrius, però si voleu podeu fer servir les vostres implementacions pròpies del chatbot i del recomanador basat en pagerank.

Si feu servir el codi donat, llegiu les consideracions que trobareu més abaix

In [34]:
import zipfile
import pandas as pd
import numpy as np
from os.path import join, dirname


def locate(*path):
    base = globals().get('__file__', '.')
    return join(dirname(base), *path)

def unzip(file):
    zip_ref = zipfile.ZipFile(locate(file), 'r')
    zip_ref.extractall(locate('data'))
    zip_ref.close()

unzip('order_products__train.csv.zip')
unzip('orders.csv.zip')
unzip('products.csv.zip')
unzip('aisles.csv.zip')

In [35]:
df_order_prods = pd.read_csv(locate('data', 'order_products__train.csv'))
df_orders = pd.read_csv(locate('data', 'orders.csv'))[['order_id', 'user_id']]
df_prods = pd.read_csv(locate('data', 'products.csv'))[['product_id', 'aisle_id']]

df_merged = pd.merge(pd.merge(df_order_prods, df_orders, on='order_id'), df_prods, on='product_id')
counts = df_merged.groupby(['user_id', 'aisle_id']).size()
df_counts = counts.unstack()

In [36]:
df_aisles = pd.read_csv(locate('data', 'aisles.csv'), index_col='aisle_id')

In [37]:
if __name__ == '__main__':
    print(df_aisles.head())

                               aisle
aisle_id                            
1              prepared soups salads
2                  specialty cheeses
3                energy granola bars
4                      instant foods
5         marinades meat preparation


## Pràctica

**Punt 1.** L'usuari solament ha de poder afegir productes que es trobin en el dataframe `df_aisles`, és a dir, el nom del producte s'ha de trobar en la columna `aisles` d'aquest dataframe

**Punt 2.** Quan l'usuari enviï la comanda `/reco` i solament si es troba afegint productes (podeu cridar `user.is_adding()` per comprovar-ho), li heu de recomanar un nou producte que no tingui ja a la llista. Per fer-ho, els passos que seguirem seran els següents:

1. Buscar a la base de dades (`df_counts`) la persona més semblant a l'usuari del bot. Això es pot fer de diverses maneres, per exemple, pots mirar quina persona té una distància més petita respecte l'usuari tenint en compte les compres, amb `np.linalg.norm(compres_persona_db - llista_usuari)`, o agant la que té la correlació de pearson més gran entre les seves compres i la de l'usuari (`scipy.stats.stats.pearsonr` o el mètode `corr` dels dataframes).

Està clar, per fer això necessites la llista de productes afegits de l'usuari en funció de `aisle_id` (no el nostre `product_id`) i de la quantitat `qty`, pots obtenir-ho a partir de `user_info(user.id)['products']`.

2. Un cop tens aquesta persona, calcula el seu `score` (ie. l'estimació de compra) per totes les `aisle_id` que l'usuari no hagi comprat encara.

3. Envia un missatge a l'usuari amb el nom de la `aisle_id` que ha tret millor puntuació en el punt anterior i la puntuació arrodonida a l'enter més proper

## Consideracions del recomanador

El recomanador es dóna ja entrenat (arxius P.pkl i Q.pkl), però de forma ràpida i poc fiable. Podeu obtenir la recomanació d'un usuari de la base de dades (df_counts) per a un item (aisle_id) donat amb el mètode `estimate(usuari, item)`.

In [38]:
from recommender import NMFRecommender

reco = NMFRecommender(df_counts, 3, 10)
reco.factorize()
reco.estimate(1, 2)

4.9759486763891934

## Consideracions del chatbot

* No teniu accés directe al codi del bot, treballareu a partir de funcions "callback", és a dir, quan el bot detecta un event cridarà les vostres funcions. Les funcions, totes elles, tenen un dos paràmetres en comú:
    * `bot`: Objecte DelegatorBot de Telepot
    * `user`: Objecte ShoppingUser
    
Les funcions que es cridaran a mode de callback són:

* Quan es rep una comanda, és a dir un missatge que comença per /, es cridarà `on_cmd(bot, user, cmd)`. El paràmetre `cmd` conté la comanda enviada per l'usuari
* Quan s'afegeix un producte, es crida `on_add(bot, user, product_id, qty)`, on `product_id` indica el nom del producte i `qty` la quantitat comprada. Si aquesta funció retorna True o None, l'item s'afegirà a l'usuari, però si retorna False **no** s'afegirà.
* Quan es marca un producte com a comprat (si encara no estava comprat), es crida `on_flag(bot, user, product_id)`
* Quan s'acaben de comprar tots els productes, i solament 1 cop per interacció, es crida `on_end(bot, user)`

**Els productes de l'usuari ja no són una llista de productes, sinó diccionari de productes**:

```python
{
    ...
    'products': {
        'product_id_1': {
            'status': 0/1,
            'qty': <int>
        },
        ...
        'product_id_n': {
            'status': 0/1,
            'qty': <int>
        },
    }
    ...
}
```

In [62]:
import numpy as np
from recommender import NMFRecommender

# we initialize the recommender
reco = NMFRecommender(df_counts, 3, 10)
reco.factorize()

async def on_cmd(bot, user, cmd):
    if cmd == '/start':
        await bot.sendPhoto(user.id, open('img/hello.jpg', 'rb'))
    elif cmd == '/done':
        await user.sender.sendMessage('Let\'s go')
    elif cmd == '/reco':
        if user.is_adding():
            # we first create  a vector with zeros to fill with the new user values        
            u2 = np.zeros(df_counts.shape[1])
            # we save the user information in a variable to reduce the calls to user_info()
            user_i = user_info(user.id)
            
            # this counter will be used to determine which aisle are whe at while itering
            i = 0
            # we transform the user information products keys to a list
            prod = list(user_i['products'].keys())
            # for each aisle in our df aisles dataframe
            for aisle in df_aisles.values.tolist():
                # if the aisle is in our user products list
                if aisle[0] in prod:
                    # we set the value to the new vector we are creating
                    u2[i] = user_i['products'][aisle[0]]['qty']
                i += 1
            # now we create a matrix with the same size as our user database
            # and we fill it with the values we obtained from the new user
            M_u2 = np.full((df_counts.shape[0],df_counts.shape[1]),u2)
            # with this matrix we calculate the norm between both matrices (in the row axis) to
            # determine which user from our database is closer to the new user
            calculated_norm = np.linalg.norm(df_counts.fillna(0).as_matrix() - M_u2,axis=1)
            # we obtain the position of the closer user and using iloc we get the id of the user in our dataframe
            recommended_user = df_counts.iloc[np.argmin(calculated_norm)].name
            # we create a list to fill with the values from the recommender
            recommended_values = []
            # for each aisle 
            for i in range(df_counts.shape[1]):
                # if the product isnt already bought
                if df_aisles.iloc[i-1].values[0] not in list(user_i['products'].keys()):
                    # we add the tuple (value, aisle_id)
                    recommended_values.append((reco.estimate(recommended_user, i),i)) 
            # if the recommended values is not empty
            if len(recommended_values):
                max_value = max(recommended_values)
                # we get the recommended aisle using the maximum value from the recommendations
                recommended_aisle = df_aisles.loc[max_value[1]]
                # we send a message to the user, with the recommended aisle and recommended quantity
                await user.sender.sendMessage("I'd recommend you to buy "+str(int(max_value[0]))+" products from: \n ---> " 
                                              + recommended_aisle.values[0] + " <---")
            # if we can't recommend anything
            else:
                # we better thank the user to buy everything we have!
                await user.sender.sendMessage("I honestly can't recommend you anything you haven't bought already.\n"+
                                             "...\n"+ 
                                              "Though, I'm happy you have bought all the things we have to offer. Thank you <3")
            #print for debug purposes
            print(recommended_user, list(user_i['products'].keys()), (max(recommended_values)))
        
        
        else:
            await user.sender.sendMessage("I'd recommend you to pay your products first, before asking me " 
                                          +"to recommend you what to buy ;)")
    # extra command, not relevant for the assignment
    elif cmd == '/sad':
            await user.sender.sendMessage('I was programmed for an assignment due the day before christmas :(')
            await bot.sendPhoto(user.id, open('img/christmas.jpg', 'rb'))
   
    
async def on_add(bot, user, product_id, qty):    
    if product_id in df_aisles['aisle'].tolist():
        return True
    else:
        return False
    
async def on_flag(bot, user, product_id):
    pass
    
async def on_end(bot, user):
    await bot.sendPhoto(user.id, open('img/done.png', 'rb'))
    pass

In [63]:
if __name__ == '__main__':
    from bot import ShoppingBot, user_info
    
    with ShoppingBot() as bot:
        # Setup callbacks
        bot.add_callback('cmd', on_cmd)
        bot.add_callback('add-product', on_add)
        bot.add_callback('flag-product', on_flag)
        bot.add_callback('end', on_end)
        
        # Start bot
        bot.start(open('TOKEN').read().strip())

KeyboardInterrupt: 